In [1]:
FEF

NameError: name 'FEF' is not defined

In [2]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud

sys.path.append('..')
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

import pandas as pd
import geopandas as gpd

import locale
import time
from datetime import datetime, timedelta

import sqlite3
from datetime import datetime
from shapely import wkb
import os

In [3]:
# CONSTANTES
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

PATH_XLSX_CONTAC = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\CONTACTOS.xlsx'

In [4]:
# conecta o crea la base de datos
conn = sqlite3.connect('mensajes_whatsapp.db')
# cursor a la base de datos
cursor = conn.cursor()
# script parta crear si no existe la tabla
cursor.execute('''
CREATE TABLE IF NOT EXISTS mensajes_whatsapp (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fecha TIMESTAMP,
    idd INTEGER,
    cod_canero INTEGER,
    nombre_canero TEXT,
    numero INTEGER,
    mensaje TEXT,
    enviado BOOLEAN DEFAULT 0,
    fecha_enviado TIMESTAMP
);
''')
# ejecura el script
conn.commit()
# cierra la conexion
conn.close()

In [5]:
# leer token
acceso = open(RUTA_COMPLETA + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
# DATOS DE PILOTOS
tecnico_telf = {
    'ALEJANDRO SANCHEZ' : 78448317,
    'AURELIO GARCIA' : 73975475,
    'GABRIEL PEÑA' : 76368561,
    'EDUARDO REYES' : 75016609}

aux_telf = {
    'JOSE ARMANDO CASANOVA' : 68908131,
    'MARIO SANCHEZ' : 75380725,
    'KARINA DE OLIVEIRA' : 77437601,
    'BISMARK SOCOMPI' : 78194371}

In [7]:
# EJECUTAR QUERY
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

# CORVERSION DE WKB A COORS
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

def get_registro_notificacion_tricho():
    try:
        # get data de nuevas notificacion de liberacion
        query = 'select *, \'tricho\' as origen from dataset_354655 where procesado=false'
        notif = ejecutar_query_sql(33457, query, 'get')
        notif = notif['data']
        # extrae el primer elemento
        return notif
    except Exception as general_err:
        print(f"Error inesperado al conectarse a AmigoCloud [def get_registro_notificacion()]: {general_err}")
        return None
    return None

def get_registro_notificacion_pulv():
    try:
        # get data de nuevas notificacion de liberacion
        query = 'select *, \'pulv\' as origen from dataset_360917 where procesado=false'
        notif = ejecutar_query_sql(35248, query, 'get')
        notif = notif['data']
        # extrae el primer elemento
        return notif
    except Exception as general_err:
        print(f"Error inesperado al conectarse a AmigoCloud [def get_registro_notificacion()]: {general_err}")
        return None
    return None

def anular_registro_notificacion(idd, origen):
    try:
        if origen == 'tricho':
            query = f'update dataset_354655 set procesado = true where id = {idd}'
            exe = ejecutar_query_sql(33457, query, 'post')
            return True
        elif origen == 'pulv':
            query = f'update dataset_360917 set procesado = true where id = {idd}'
            exe = ejecutar_query_sql(35248, query, 'post')
            return True
    except Exception as general_err:
        print(f"Error inesperado: {general_err}")
        return False
    return None


def registrar_nuevo_msj(fecha, idd, cod_ca, nom_ca, num_cell, msj):
    intentos = 5  # cantidad máxima de intentos si la DB está bloqueada
    espera = 10  # segundos de espera entre intentos

    for intento in range(intentos):
        try:
            with sqlite3.connect('mensajes_whatsapp.db', timeout=10) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                    INSERT INTO mensajes_whatsapp (fecha, idd, numero, cod_canero, nombre_canero, mensaje)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (fecha, idd, cod_ca, nom_ca, num_cell, msj))
                conn.commit()
                print("✅ Mensaje registrado exitosamente. id: " + str(idd) + ", para: " + nom_ca)
                break  # sale del loop si se inserta correctamente
        except sqlite3.OperationalError as e:
            print(f"⚠️ Intento {intento} - Registro id: {str(idd)} - Base bloqueada: {e}")
            if intento < intentos:  # si no es el último intento, espera
                time.sleep(espera)
            else:
                print("❌ Error: no se pudo registrar después de múltiples intentos. id: " + str(idd))
    return None

def get_nums_cells_validos(cod_ca):
    df_contac = pd.read_excel(PATH_XLSX_CONTAC)
    df_contac.fillna(0, inplace=True)
    df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]
    contac_filtro = df_contac[df_contac['cod_ca']==cod_ca]
    
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        return telefonos_validos
    else:
        return [] 
    return None

def generar_msj_para_coordinador(fecha, idd, msj):
    registrar_nuevo_msj(fecha, idd, 0, 'JOSE ARMANDO CASANOVA', 68908131, msj)
    return None

def generar_msj_isnull(df_notif_isnull):
    for i, row in df_notif_isnull.iterrows():
        fecha = row['fecha_registro']
        idd = row['id']
        piloto = row['piloto']
        canhero = row['canhero']
        tipo_labor = row['tipo_labor']
        tipo_mensaje = row['tipo_mensaje']
    
        msj = f'{piloto}, has registrado una notificaion de {tipo_mensaje} de {tipo_labor} sin indicar el cañero, sin nombre del cañero la notificacion no se enviará, vuelve abrir el punto de la notificacion creada y define el cañero'

        if piloto in tecnico_telf:
            num_cell = tecnico_telf[piloto]
            registrar_nuevo_msj(fecha, idd, 0, piloto, num_cell, msj)
            
            # mensaje para coordinador
            msj = f'{piloto} ha registrado una notificaion de {tipo_mensaje} de {tipo_labor} sin indicar el cañero, sin nombre del cañero la notificacion no se enviará'
            generar_msj_para_coordinador(fecha, idd, msj)
        else:
            print('El nombre de piloto no esta registrado')
            continue
    return None   

def extraer_fecha_hora(fecha):
    # Establecer el idioma español
    locale.setlocale(locale.LC_TIME, "Spanish_Spain.1252")  # Para Windows
    
    dt = datetime.fromisoformat(fecha)
    # Restar 4 horas
    dt_modificado = dt - timedelta(hours=4)

    # Extraer fecha y hora en formato texto
    fecha_str = dt_modificado.strftime("%d de %B de %Y")
    hora_str = dt_modificado.strftime("%I:%M:%S%p")
    return fecha_str, hora_str

def generar_msj_notnull(df_notif_notnull):
    for i, row in df_notif_notnull.iterrows():
        fecha = row['fecha_registro']
        idd = row['id']
        piloto = row['piloto']
        cod_ca = int(row['canhero'].split(' / ')[0])
        canhero = row['canhero'].split(' / ')[1]
        tipo_labor = row['tipo_labor']
        tipo_mensaje = row['tipo_mensaje']
        ubicacion = convertir_wkb(row['ubicacion'])
        longitud = ubicacion.x  # Longitud (coordenada X)
        latitud = ubicacion.y   # Latitud (coordenada Y)
        origen = row['origen']
        
        fecha_str, hora_str = extraer_fecha_hora(fecha)
        print(fecha_str, hora_str)
        
        if piloto in tecnico_telf == False:
            print('El nombre de piloto no esta registrado')
            continue ## NOTIFICAR AL COORDINADOR QUE EL PILOTO NO ESTA EN LA LISTA DE PILOTOS
        
        num_cell = tecnico_telf[piloto]
        msj = f'''📢 *Mensaje generado automáticamente por el equipo de campo* 📢

👤 Señor(a) *{canhero}*, espero que se encuentre bien.        

🛠️ *Se le informa sobre la siguiente actividad:*
- 🏷️ Estado: *{tipo_mensaje}*
- 🚁 Labor: *{tipo_labor} con drones*
- 📍 Ubicación: [📌 Ver en mapa](https://www.google.com/maps?q={latitud},{longitud})
- 📅 Fecha: *{fecha_str}*
- ⏰ Hora: *{hora_str}*

📞 Para cualquier consulta puede comunicarse con el piloto responsable:
👨‍✈️ *ING. {piloto}* al 📲 *{num_cell}*
'''
        
        nums_validos = get_nums_cells_validos(cod_ca)
        if (len(nums_validos) == 0):
            msj = f'''{piloto} ha registrado {tipo_mensaje} para el trabajo de {tipo_labor} para el cañero {canhero}, sin embargo, el cañero no tiene numeros validos registrados en la lista de contactos'''
            generar_msj_para_coordinador(fecha, idd, msj)
            continue
        
        if anular_registro_notificacion(idd, origen) == False:
            print('Error al anular una notificacion..!!!')
            continue
        
        # mensaje para el piloto
        registrar_nuevo_msj(fecha, idd, 0, piloto, num_cell, msj)
        
        # registra mensajes para los numeros validos de cañero
        for num in nums_validos:
            registrar_nuevo_msj(fecha, idd, cod_ca, canhero, num, msj)
            
        # registra mensajes para los numeros auxliares
        for aux in aux_telf:
            registrar_nuevo_msj(fecha, idd, 0, aux, aux_telf[aux], msj)
    return None

def procesar_notificaciones():
    # obtener notificaciones nuevas
    notif_tricho = get_registro_notificacion_tricho()
    if notif_tricho == None:
        print('Error al ejecuar: def get_registro_notificacion()')
    # convertir a dataframe

    notif_pulv = get_registro_notificacion_pulv()
    if notif_pulv == None:
        print('Error al ejecuar: def get_registro_notificacion_2()')
    # convertir a dataframe

    notif = notif_tricho + notif_pulv
    
    df_notif = pd.DataFrame(notif)
    # se tienen nuevas notificaciones
    if len(df_notif) > 0:
        # filtrar registros con canero null
        df_notif_isnull = df_notif[df_notif['canhero'].isnull()]
        # filtrar registros con canero notnull
        df_notif_notnull = df_notif[df_notif['canhero'].notnull()]
        # procesar notificaciones con canero null
        if len(df_notif_isnull) > 0:
            generar_msj_isnull(df_notif_isnull)
        # procesar notificaciones con canero notnull
        if len(df_notif_notnull) > 0:
            generar_msj_notnull(df_notif_notnull)
    else:
        print('No existen nuevas notificaciones')
    return None

In [ ]:
while True:
    procesar_notificaciones()
    time.sleep(600)  # 600 segundos = 10 minutos

# Eliminar datos de tabla

In [ ]:
conn = sqlite3.connect('mensajes_whatsapp.db')
cursor = conn.cursor()
# Borrar todos los mensajes de la tabla (pero no elimina la tabla)
cursor.execute('DELETE FROM mensajes_whatsapp;')
# Reiniciar el contador del ID autoincremental (opcional)
cursor.execute('DELETE FROM sqlite_sequence WHERE name="mensajes_whatsapp";')
conn.commit()
conn.close()

# Imprimir datos de tabla

In [8]:
conn = sqlite3.connect('mensajes_whatsapp.db')
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM mensajes_whatsapp WHERE enviado = 0;
''')
pendientes = cursor.fetchall()
for msg in pendientes:
    print(msg)
conn.close()

(63, '2025-07-14 22:16:45+00:00', 518, 'EDUARDO REYES', '75016609', 0, 'EDUARDO REYES, has registrado una notificaion de INICIO DE LABOR de LIBERACION DE TRICHOGRAMMA sin indicar el cañero, sin nombre del cañero la notificacion no se enviará, vuelve abrir el punto de la notificacion creada y define el cañero', 0, None)
(64, '2025-07-14 22:16:45+00:00', 518, 'JOSE ARMANDO CASANOVA', '68908131', 0, 'EDUARDO REYES ha registrado una notificaion de INICIO DE LABOR de LIBERACION DE TRICHOGRAMMA sin indicar el cañero, sin nombre del cañero la notificacion no se enviará', 0, None)
(65, '2025-07-14 22:23:42+00:00', 519, 'EDUARDO REYES', '75016609', 0, 'EDUARDO REYES, has registrado una notificaion de FIN DE LABOR de LIBERACION DE TRICHOGRAMMA sin indicar el cañero, sin nombre del cañero la notificacion no se enviará, vuelve abrir el punto de la notificacion creada y define el cañero', 0, None)
(66, '2025-07-14 22:23:42+00:00', 519, 'JOSE ARMANDO CASANOVA', '68908131', 0, 'EDUARDO REYES ha regis

In [ ]:
registrar_nuevo_msj('2025-03-17', 1, 1, 'BISMARK SOCOMPI', 78194371, 'hola 1')
registrar_nuevo_msj('2025-03-17', 1, 1, 'BISMARK SOCOMPI', 78194371, 'hola 2')
registrar_nuevo_msj('2025-03-17', 1, 1, 'BISMARK SOCOMPI', 78194371, 'hola 3')